In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes
from matplotlib.spines import Spine
from matplotlib import cm

import numpy as np
import numpy.typing as npt

import os
import csv

### ax.plot(y)

In [14]:
# 하나의 리스트를 이용하여 그래프 생성
np.random.seed(0)

y_data = np.random.normal(loc=0, scale=1, size=(300,))

fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(y_data)

fig.tight_layout()
ax.tick_params(labelsize=15)

# grid와 spine 설정
ax.grid()
spine: Spine
for spine_loc, spine in ax.spines.items():
    spine.set_linewidth(3)

    if spine_loc in ['right', 'top']:
        spine.set_visible(False)

    # spine의 위치를 0으로 이동
    if spine_loc in ['bottom', 'left']:
        spine.set_position(('data', 0))

In [15]:
# x ticks 설정
y_data = np.random.normal(loc=0, scale=1, size=(300,))

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(y_data)

fig.tight_layout()
ax.tick_params(labelsize=25)

x_ticks = np.arange(301, step=100)
ax.set_xticks(x_ticks)

In [3]:
def bop_data_reader() -> np.array:
    month_list: list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # 'Jan': 1, 'Feb': 2 ...
    month_dict = {month_list[m_idx]: m_idx + 1 for m_idx in range(len(month_list))}
    dataset = list()

    # 현재 위치에 있는 파일을 읽는다. 패키지 내에서 실행될 수 있으므로
    # file_name = os.path.join(os.path.dirname(__file__), 'BrentOilPrices.csv')
    file_name = os.path.join(os.getcwd(), 'BrentOilPrices.csv')
    with open(file_name, 'r', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)     # 헤더를 제외한다.

        for data in csvreader:
            date: str = data[0]
            price: float = float(data[-1])

            # Year, Month, Day 순으로 맞춘다.
            year: int
            month: int
            day: int
            if '-' in date:
                tokens = date.split('-')
                year = int(tokens[-1])
                # year가 두 자리이므로
                year = 1900 + year if year > 50 else 2000 + year
                month = month_dict.get(tokens[1])
                day = int(tokens[0])
            elif ',' in date:
                tokens = date.replace(',', ' ').split()
                year = int(tokens[-1])
                month = month_dict.get(tokens[0])
                day = int(tokens[1])
            else:
                continue

            dataset.append([year, month, day, price])

    # 전체 dataset을 float로 변환
    dataset: np.array = np.array(dataset).astype(np.float_)
    return dataset


def get_year_data(dataset: np.array, t_year: float) -> np.array:
    # 지정한 년도의 데이터만 추출한다.
    t_idx: np.array = np.where(dataset[:, 0] == t_year)
    t_data: np.array = dataset[t_idx]
    return t_data

In [16]:
dataset = bop_data_reader()
t_year = 1990
t_year_data = get_year_data(dataset, t_year)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t_year_data[:, -1])

ax.set_title("BOP data {}".format(t_year), fontsize=15)
ax.set_ylabel("daily prices in USD", fontsize=15)
ax.tick_params(labelsize=10)
ax.grid(axis='y')

### ax.plot(x, y)

In [17]:
np.random.seed(0)

n_data = 100
s_idx = 30
x_data = np.arange(s_idx, s_idx + n_data)
y_data = np.random.normal(0, 1, (n_data,))

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x_data, y_data)

fig.tight_layout(pad=3)
x_ticks = np.arange(s_idx, s_idx + n_data + 1, 20)
ax.set_xticks(x_ticks)

ax.tick_params(labelsize=20)
ax.grid()

In [18]:
# 임의 간격의 그래프 생성
np.random.seed(0)

x_data = np.array([10, 25, 31, 40, 55, 80, 100])
y_data = np.random.normal(0, 1, len(x_data,))

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x_data, y_data)

fig.subplots_adjust(left=0.2)
ax.tick_params(labelsize=20)

# x축과 y축의 tick을 설정
ax.set_xticks(x_data)
ylim = ax.get_ylim()
y_ticks = np.linspace(ylim[0], ylim[-1], 8)
ax.set_yticks(y_ticks)

ax.grid()


In [19]:
# 무작위 데이터를 이용하여 라인 그래프 생성
np.random.seed(0)

x_data = np.random.normal(0, 1, (10,))
y_data = np.random.normal(0, 1, (10,))

fig, ax = plt.subplots()
ax.plot(x_data, y_data)

In [21]:
# 각 월의 첫번째 날의 데이터를 이용하여 그래프 생성
dataset = bop_data_reader()
t_year = 1990
t_year_data = get_year_data(dataset, t_year).astype(np.int_)

fig, ax = plt.subplots(figsize=(10, 7))

# 각 월의 첫번째 날짜를 찾는다.
month_dict = {m: None for m in range(1, 1 + 12)}

for data_idx, data in enumerate(t_year_data):
    Y, M, D, price = data

    if month_dict[M] is None:
        month_dict[M] = [data_idx, '-'.join(str(i) for i in (Y, M, D))]

first_day_label = np.array(list(month_dict.values()))
# ------------

x_arange = np.arange(t_year_data.shape[0])
ax.plot(x_arange, t_year_data[:, -1])  # 모든 행의 마지막 열의 데이터가 가격

ax.set_xticks(first_day_label[:, 0].astype(np.int_))
ax.set_xticklabels(first_day_label[:, 1], rotation=-30, ha='left')

ax.set_title("BOP data {}".format(t_year), fontsize=20)
ax.set_ylabel("daily price in USD", fontsize=15)
ax.tick_params(labelsize=10)
ax.grid()

### Several Line Plots on One Ax

In [41]:
n_data = 100

random_noise1 = np.random.normal(0, 1, (n_data,))
random_noise2 = np.random.normal(1, 1, (n_data,))
random_noise3 = np.random.normal(2, 1, (n_data,))

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(random_noise1)
ax.plot(random_noise2)
ax.plot(random_noise3)

ax.tick_params(labelsize=15)

In [42]:
# 데이터의 개수가 다른 여러 개의 그래프 그리기
n_data1, n_data2, n_data3 = 200, 50, 10

x_data1 = np.linspace(0, 200, n_data1)
x_data2 = np.linspace(0, 200, n_data2)
x_data3 = np.linspace(0, 200, n_data3)

random_noise1 = np.random.normal(0, 1, (n_data1,))
random_noise2 = np.random.normal(1, 1, (n_data2,))
random_noise3 = np.random.normal(2, 1, (n_data3,))

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(x_data1, random_noise1)
ax.plot(x_data2, random_noise2)
ax.plot(x_data3, random_noise3)

ax.tick_params(labelsize=15)

In [43]:
t = np.linspace(-4 * np.pi, 4 * np.pi, 300)
sin = np.sin(t)
linear = 0.1 * t

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(t, sin)
ax.plot(t, linear)

ax.set_ylim([-1.5, 1.5])

x_ticks = np.arange(-4* np.pi, 4 * np.pi + 0.1, np.pi)
x_ticklabels = [str(i) + r'$\pi$' for i in range(-4, 5)]

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticklabels)

ax.tick_params(labelsize=15)
ax.grid()

In [44]:
# BOP data plotting

def bop_plot(dataset, t_year: int, ax: Axes):
    t_year_data = get_year_data(dataset, t_year)
    ax.plot(t_year_data[:, -1])

dataset = bop_data_reader()

fig, ax = plt.subplots(figsize=(8, 6))

bop_plot(dataset, 1990, ax)
bop_plot(dataset, 1993, ax)
bop_plot(dataset, 1996, ax)

ax.set_title("BOP data", fontsize=20)
ax.set_ylabel("daily prices in USD", fontsize=15)
ax.tick_params(labelsize=15)
ax.grid()

### Several Line Plots on Different Axes

In [45]:
t = np.linspace(-4 * np.pi, 4 * np.pi, 1000)

sin = np.sin(t)
cos = np.cos(t)
tan = np.tan(t)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7, 7))

axes[0].plot(t, sin)
axes[1].plot(t, cos)
axes[2].plot(t, tan)

axes[2].set_ylim([-5, 5])

fig.tight_layout()

In [46]:
t = np.linspace(-4 * np.pi, 4 * np.pi, 1000).reshape(1, -1)
sin = np.sin(t)
cos = np.cos(t)
tan = np.tan(t)

data = np.vstack((sin, cos, tan))

title_list = [r'$sin(t)$', r'$cos(t)$', '$tan(t)$']
x_ticks = np.arange(-4 * np.pi, 4 * np.pi + np.pi, np.pi)
x_ticklabels = [str(i) + r'$\pi$' for i in range(-4, 5)]

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7, 7), sharex=True)

ax: Axes
for ax_idx, ax in enumerate(axes.flat):
    ax.plot(t.flatten(), data[ax_idx])
    ax.set_title(title_list[ax_idx], fontsize=20)
    ax.tick_params(labelsize=15)
    ax.grid()
    if ax_idx == 2:
        ax.set_ylim([-3, 3])

fig.subplots_adjust(left=0.1, right=0.95, bottom=0.05, top=0.95)
axes[-1].set_xticks(x_ticks)
axes[-1].set_xticklabels(x_ticklabels)

[Text(-12.566370614359172, 0, '-4$\\pi$'),
 Text(-9.42477796076938, 0, '-3$\\pi$'),
 Text(-6.283185307179586, 0, '-2$\\pi$'),
 Text(-3.141592653589793, 0, '-1$\\pi$'),
 Text(0.0, 0, '0$\\pi$'),
 Text(3.141592653589793, 0, '1$\\pi$'),
 Text(6.283185307179586, 0, '2$\\pi$'),
 Text(9.42477796076938, 0, '3$\\pi$'),
 Text(12.566370614359172, 0, '4$\\pi$')]

In [47]:
# BOP 데이터를 여러 축에 그리기
def bop_plot(dataset, t_year, ax):
    t_year_data = get_year_data(dataset, t_year).astype(np.int_)

    month_dict = {m: None for m in range(1, 12 + 1)}
    for data_idx, data in enumerate(t_year_data):
        Y, M, D, data = data
        if month_dict[M] is None:
            month_dict[M] = [data_idx, '-'.join(str(i) for i in (Y, M, D))]

    first_day_label = np.array(list(month_dict.values()))

    x_arange = np.arange(t_year_data.shape[0])
    ax.plot(x_arange, t_year_data[:, -1])
    ax.set_xticks(first_day_label[:, 0].astype(np.int_))
    ax.set_xticklabels(first_day_label[:, 1], rotation=-30, ha='left')
    ax.set_title("BOP data {}".format(t_year), fontsize=15)

    ax.tick_params(labelsize=15)
    ax.grid()
# ----------

dataset = bop_data_reader()
t_year_list = [1990, 1991, 1995]
fig, axes = plt.subplots(len(t_year_list), 1, figsize=(12, 8))

for ax_idx, ax in enumerate(axes.flat):
    bop_plot(dataset, t_year_list[ax_idx], ax)

    if ax_idx == 1:
        ax.set_ylabel("daily prices in USD", fontsize=15)

fig.tight_layout()

### ax.axvline and ax.axhline

In [48]:
fig, ax = plt.subplots()

ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])

ax.axvline(x = 1, color='black', linewidth=1)

In [49]:
fig, ax = plt.subplots()

ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])

ax.axvline(x = 1,
           ymax=0.8, ymin=0.2,
           color='black', linewidth=1)

In [50]:
fig, ax = plt.subplots()

ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])

ax.axhline(y = 1, color='black', linewidth=1)

In [51]:
fig, ax = plt.subplots()

ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])

ax.axhline(y = 1,
           xmax=0.8, xmin=0.2,
           color='black', linewidth=1)